In [1]:
from geebap import scores

In [2]:
import ee

In [3]:
from geetools import tools, ui, cloud_mask, algorithms, collection

In [4]:
Map = ui.Map()

In [5]:
Map.show()

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

In [6]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

Masked image

In [7]:
i = ee.Image('COPERNICUS/S2/20181122T142749_20181122T143353_T18GYT')

In [8]:
s2col = collection.Sentinel2TOA()

In [9]:
s2col.bit_options()

{'QA60': ['cirrus', 'cloud']}

In [10]:
masked = s2col.apply_mask(i, 'QA60', ['cloud'])
masked = masked.clip(aoi)

In [11]:
vis = {'bands':['B8', 'B11', 'B4'], 'min':0, 'max':5000}

In [12]:
Map.addLayer(masked, vis, 'Masked')

In [13]:
Map.centerObject(masked)

## CloudDist score

In [14]:
cld = scores.CloudDist()

In [15]:
cldband = cld.compute(masked, bandmask='B2', dmin=0, dmax=600)

In [16]:
Map.addLayer(cldband, {'min':0, 'max':1, 'palette':['white', 'red', 'orange', 'yellow', 'green', 'blue', 'black']}, 'Cloud Distance Score')

## Using collection module

In [17]:
masked = s2col.rename(masked)

In [18]:
icol = ee.ImageCollection.fromImages([masked])

In [19]:
col_score = cld.map(icol, col=s2col)

In [20]:
newi = ee.Image(col_score.first())

In [21]:
Map.addLayer(newi, {'bands':[cld.name], 'min':0, 'max':1}, 'Cloud Distance Score from satcol')